In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet import MobileNet
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.initializers import glorot_uniform
from keras.layers import (Activation, BatchNormalization, Conv2D, Dense,
                          Dropout, Flatten, MaxPooling2D)
from keras.losses import binary_crossentropy
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from matplotlib import image
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix


Using TensorFlow backend.


In [2]:
SEED = 1234
model = Sequential()
input_shape = (224, 224, 3)
model.add(Conv2D(16, (3, 3), input_shape=input_shape, kernel_initializer=glorot_uniform(seed=SEED)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(32, (3, 3), kernel_regularizer=l2(0.01), kernel_initializer=glorot_uniform(seed=SEED)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), kernel_regularizer=l2(0.01), kernel_initializer=glorot_uniform(seed=SEED)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(rate=0.3, seed=SEED))

model.add(Flatten())
model.add(Dense(512, kernel_regularizer=l2(0.01), kernel_initializer=glorot_uniform(seed=SEED)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1, activation='sigmoid', kernel_initializer=glorot_uniform(seed=SEED)))

model.compile(optimizer=SGD(lr=0.01, nesterov=True),
              loss=binary_crossentropy,
              metrics=['accuracy'])

In [3]:
from keras.models import load_model

In [4]:
model.load_weights('weights.h5')

In [5]:
import sys
v_path = '/opt/ros/kinetic/lib/python2.7/dist-packages'
if v_path in sys.path:
    sys.path.remove(v_path)
import numpy as np
import pandas as pd
from keras.applications.resnet50 import ResNet50
from keras.optimizers import SGD, Adam
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense
from PIL import Image  
import os
import csv
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
def make_square(fp):
    image = Image.open(fp)  
    image = image.resize((224, 224))
    r_img = np.array(image)
    if(r_img.shape == (224, 224)):
        return r_img.reshape(224, 224, 1)
    return r_img[:,:,0].reshape(224, 224, 1)

In [7]:
# Testing
import os
X_test = []
index = []
for i in range(301):
    print(i)
    fp1 = 'TestData(XraysOnly)/Test (' + str(i+1) + ')/AP/AP.jpg'
    fp2 = 'TestData(XraysOnly)/Test (' + str(i+1) + ')/LAT/LAT.jpg'
    if(os.path.exists(fp1) and os.path.exists(fp2)):
        img1 = make_square(fp1)
        img2 = make_square(fp2)
        datapoint = np.append(img1, img2, axis=2)
        X_test.append(datapoint)
        index.append(i+1)
X_test = np.array(X_test)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
X1 = np.zeros((X_test.shape[0], 224, 224)).reshape(X_test.shape[0], 224, 224, 1)

In [9]:
X_test = np.append(X_test, X1, axis=3)

In [10]:
y = model.predict(X_test)

In [11]:
final_dict = {}

In [12]:
for i in range(y.shape[0]):
    if(y[i] >= 0.5 ):
        y[i] = 1
    else:
        y[i] = 0
    final_dict[index[i]] = y[i]

In [14]:
import pickle
pickle_out = open("test.pickle","wb")
pickle.dump(final_dict, pickle_out)
pickle_out.close()

In [16]:
import numpy as np

In [18]:
y_seg1 = np.load('results/y_seg1_test.npy')
y_seg2 = np.load('results/y_seg2_test.npy')
y_seg3 = np.load('results/y_seg3_test.npy')
y_seg4 = np.load('results/y_seg4_test.npy')
y_seg5 = np.load('results/y_seg5_test.npy')
y_seg6 = np.load('results/y_seg6_test.npy')
y_seg7 = np.load('results/y_seg7_test.npy')
y_seg8 = np.load('results/y_seg8_test.npy')

In [19]:
y_seg1.shape

(299, 224, 224)

In [20]:
dict1 = {}
for i in range(y_seg1.shape[0]):
    dict2 = {}
    dict2['Ap_Pedicle'] = y_seg1[i]
    dict2['Ap_Spinous_Process'] = y_seg2[i]
    dict2['Ap_Vertebra'] = y_seg3[i]
    dict2['Lat_Anterior_Vertebral_Line'] = y_seg4[i]
    dict2['Lat_Disk_Height'] = y_seg5[i]
    dict2['Lat_Posterior_Vertebral_Line'] = y_seg6[i]
    dict2['Lat_Spinous_Process'] = y_seg7[i]
    dict2['Lat_Vertebra'] = y_seg8[i]
    dict1[index[i]] = dict2

In [22]:
y_seg1[0].max()

True

In [21]:
dict1

{1: {'Ap_Pedicle': array([[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]]),
  'Ap_Spinous_Process': array([[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]]),
  'Ap_Vertebra': array([[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False,

In [23]:
import pickle
pickle_out = open("segmentation.pickle","wb")
pickle.dump(dict1, pickle_out)
pickle_out.close()

In [26]:
np.save('index', index)

In [ ]:
# with open('output1.csv', 'w', newline='') as file:
#     fieldnames = ['Image ID', 'Label']
#     writer = csv.DictWriter(file, fieldnames=fieldnames)
#     writer.writeheader()
#     for i in range(X_test.shape[0]):
#         writer.writerow({'Image ID': index[i], 'Label': y[i][0]})

In [ ]:
import os
y_given = []

for i in range(301):
    print(i)
    fp1 = 'TestData(XraysOnly)/Test (' + str(i+1) + ')/AP/AP.jpg'
    fp2 = 'TestData(XraysOnly)/Test (' + str(i+1) + ')/LAT/LAT.jpg'
    if(os.path.exists(fp1) and os.path.exists(fp2)):
        img1 = make_square(fp1)
        img2 = make_square(fp2)
        datapoint = np.append(img1, img2, axis=2)
        X_test.append(datapoint)
        index.append(i+1)
X_test = np.array(X_test)
y_given = 